# SentiArt auslesen
Zuerst importieren wir die benötigten Module.

In [2]:
import pandas as pd
import re
import os
from collections import Counter

Dann geben wir die Dateipfade an und laden SentiArt in einen Dataframe. Damit wir schneller auf die Daten zugreifen können, erstellen wir ein Dictionary mit der Spalte "word" als Index.
Anschließend stellen wir Counter, um die Effizienz des Scripts zu prüfen.

In [14]:
# Liste der Ordner, die durchlaufen werden sollen
folder_paths = [
    ('C:/Users/jwieb/Documents/Korpora_FINAL/Schauerkorpus_AKTUELL', 'Schauerkorpus'),
    ('C:/Users/jwieb/Documents/Korpora_FINAL/DTA_FINAL', 'Vergleichskorpus')
]

csv_file_path = 'C:/Users/jwieb/OneDrive - uni-bielefeld.de/Bachelorarbeit/120kSentiArt_DE.csv'
output_file_path = 'C:/Users/jwieb/OneDrive - uni-bielefeld.de/Bachelorarbeit/test_output.csv'

df = pd.read_csv(csv_file_path, sep=';')

# Wörterbuch erstellen
word_dict = {word.lower(): row for word, row in df.set_index('word').iterrows()}

columns = df.columns.tolist() + ['filename', 'subcorpus']

# Erstelle eine Liste, um die Zeilen für den neuen DataFrame zu sammeln
result_rows = []

# Zähler für gefundene und nicht gefundene Wörter
found_counter = Counter()
not_found_counter = Counter()

Jetzt iterieren wir über jeden Text des Ordners. Wenn das Wort eines Textes im Dictionary gefunden wurde, wird die Zeile zu result_rows hinzugefügt und der Counter für gefundene Wörter um eins weitergestellt. Wenn es nicht gefunden wurde, wird in allen Spalten NaN gespeichert und der Counter für nicht gefundene Wörter um eins weitergestellt. Am Ende der Schleife wird result_rows zu einem neuen Dataframe zusammengefügt.

In [15]:
for folder_path, subcorpus_name in folder_paths:
    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(folder_path, filename)

            # Lese den Text aus der aktuellen txt-Datei
            with open(file_path, 'r', encoding='utf-8') as file:
                text = file.read()

            words = re.findall(r'\b\w+\b', text.lower())

            # Iteriere durch die Wörter des Textes und sammle die Zeilen
            for word in words:
                if word in word_dict:
                    row = word_dict[word].to_dict()
                    row['filename'] = filename 
                    row['subcorpus'] = subcorpus_name
                    result_rows.append(row)
                    found_counter[word] += 1
                else:
                    not_found_entry = dict.fromkeys(columns, 'NaN')
                    not_found_entry['wordLC'] = word
                    not_found_entry['filename'] = filename
                    not_found_entry['subcorpus'] = subcorpus_name
                    result_rows.append(not_found_entry)
                    not_found_counter[word] += 1

# Erstelle den neuen DataFrame aus den gesammelten Zeilen
result_df = pd.DataFrame(result_rows)

# Entferne die 'word'-Spalte, falls vorhanden
if 'word' in result_df.columns:
    result_df.drop('word', axis=1, inplace=True)

In [16]:
result_df

,wordLC,AAPz,ang_z,fear_z,disg_z,hap_z,sad_z,surp_z,filename,subcorpus
0,zeit,0.5533,0.256,0.927,-0.423,1.3981,0.6368,0.8837,A_AlbrechtGespenst1797.txt,Schauerkorpus
1,vergangenheit,-0.389,0.274,1.0447,-0.169,-0.109,1.0997,0.6369,A_AlbrechtGespenst1797.txt,Schauerkorpus
2,überdenken,-0.47,0.231,0.1854,-1.0,-0.688,-0.578,-0.974,A_AlbrechtGespenst1797.txt,Schauerkorpus
3,blutig,-1.72,0.406,0.9858,1.4559,-0.109,0.2993,-0.974,A_AlbrechtGespenst1797.txt,Schauerkorpus
4,schlacht,-0.327,0.331,0.3973,0.1544,0.3972,0.5115,0.8447,A_AlbrechtGespenst1797.txt,Schauerkorpus
...,...,...,...,...,...,...,...,...,...,...
1021120,enthalten,-0.02,0.087,-1.886,-0.991,-1.858,-1.379,-0.74,B_ZschokkeWalpurgisnacht1812.txt,Schauerkorpus
1021121,lehren,0.3281,0.209,0.6445,0.1544,0.9881,-0.038,-1.246,B_ZschokkeWalpurgisnacht1812.txt,Schauerkorpus
1021122,kennst,-0.245,0.252,0.3266,0.3207,0.1199,0.6561,-0.246,B_ZschokkeWalpurgisnacht1812.txt,Schauerkorpus
1021123,herr,0.2052,0.2,-0.132,-0.237,0.8796,0.0582,0.0785,B_ZschokkeWalpurgisnacht1812.txt,Schauerkorpus


Jetzt prüfen wir, ob und wie das Script seinen Dienst tut.

In [17]:
print(f"Gefundene Wörter: {sum(found_counter.values())}")
print(f"Nicht gefundene Wörter: {sum(not_found_counter.values())}")

print(f"Unterschiedliche gefundene Wörter: {len(found_counter)}")
print(f"Unterschiedliche nicht gefundene Wörter: {len(not_found_counter)}")

Gefundene Wörter: 852828
Nicht gefundene Wörter: 168297
Unterschiedliche gefundene Wörter: 26299
Unterschiedliche nicht gefundene Wörter: 46579


Nun können wir die Ergebnisse in einer csv-Datei speichern.

In [8]:
filtered_df = result_df[result_df['subcorpus'] == 'Vergleichskorpus']

In [9]:
filtered_df

,wordLC,AAPz,ang_z,fear_z,disg_z,hap_z,sad_z,surp_z,filename,subcorpus
1021125,gabe,0.9014,0.225,0.6327,0.5556,2.0975,1.4951,1.2474,arnima_invalide_1818.txt,Vergleichskorpus
1021126,milde,0.4509,0.255,-0.686,0.1641,-0.471,0.8972,0.3902,arnima_invalide_1818.txt,Vergleichskorpus
1021127,bändch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,arnima_invalide_1818.txt,Vergleichskorpus
1021128,beitrag,0.6762,0.092,-1.616,-0.863,-0.893,-1.552,-1.259,arnima_invalide_1818.txt,Vergleichskorpus
1021129,l,0.5124,0.104,-1.145,-1.235,-0.881,-1.292,-1.207,arnima_invalide_1818.txt,Vergleichskorpus
...,...,...,...,...,...,...,...,...,...,...
3515783,goldfritzel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,wallenrodt_fritz02_1800.txt,Vergleichskorpus
3515784,kind,-0.245,0.269,0.974,0.536,0.3128,1.0226,-0.155,wallenrodt_fritz02_1800.txt,Vergleichskorpus
3515785,bein,-0.88,0.285,0.8328,0.399,0.3249,0.4054,-0.506,wallenrodt_fritz02_1800.txt,Vergleichskorpus
3515786,ausschließender,NaN,NaN,NaN,NaN,NaN,NaN,NaN,wallenrodt_fritz02_1800.txt,Vergleichskorpus
